In [3]:
import sys
import os
from timeit import default_timer as timer
os.getcwd()

'/home/jupyter/Notebooks/crystal/NLP/nlp_testing'

In [ ]:
# Adapted from the tutorial at https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/

In [4]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
# list(tokenizer.vocab.keys())[5000:5020]
list(tokenizer.vocab.keys())[17733]

'disgusted'

In [ ]:
# Define a new example sentence with multiple meanings of the word "disgusted"
text = "[CLS] She made a disgusted pout [SEP] Her disgusted expression was contagious [SEP]"

# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

In [ ]:
# Recreate vocabulary words from their tokenized representations.
for t in tokenized_text:
    this_word = ''
    for token in t:
        this_word += token.strip('#')
#     print(this_word)

In [ ]:
# Mark each of the tokens as belonging to sentence "0" or "1".

segments_ids = [1] * len(tokenized_text[3])
# segments_ids = [0,0,0]
print (segments_ids)
print(indexed_tokens)
print(tokenized_text[3])

In [ ]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens[3]])
segments_tensors = torch.tensor([segments_ids])

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [ ]:
# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)

In [ ]:
print ("Number of layers:", len(encoded_layers))
layer_i = 0

print ("Number of batches:", len(encoded_layers[layer_i]))
batch_i = 0

print ("Number of tokens:", len(encoded_layers[layer_i][batch_i]))
token_i = 1

print ("Number of hidden units:", len(encoded_layers[layer_i][batch_i][token_i]))

In [ ]:
# For our token, select its feature values from layer 5.
token_i = 1
layer_i = 5
vec = encoded_layers[layer_i][batch_i][token_i]
# print(vec)

# Plot the values as a histogram to show their distribution.
plt.figure(figsize=(10,10))
plt.hist(vec, bins=200)
plt.show()

In [ ]:
# `encoded_layers` is a Python list.
print('     Type of encoded_layers: ', type(encoded_layers))

# Each layer in the list is a torch tensor.
print('Tensor shape for each layer: ', encoded_layers[0].size())

In [ ]:
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(encoded_layers, dim=0)

token_embeddings.size()

In [ ]:
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

In [ ]:
# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

In [23]:
# Concatenate the last 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat_last = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat_last.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat_last), len(token_vecs_cat_last[0])))
print(token_vecs_cat_last[0])

Shape is: 1 x 3072
tensor([-0.2373,  0.8259, -0.6190,  ..., -0.3836, -0.5039,  0.6153])


In [24]:
# Sum the last 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum_last = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum_last.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum_last), len(token_vecs_sum_last[0])))

Shape is: 1 x 768


In [ ]:
# Concatenate the first 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat_first = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[0], token[1], token[2], token[3]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat_first.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat_first), len(token_vecs_cat_first[0])))
print(token_vecs_cat[0])

In [ ]:
# Sum the first 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum_first = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[:4], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum_first.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum_first), len(token_vecs_sum_first[0])))

In [ ]:
# Concatenate the middle 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat_middle1 = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[4], token[5], token[6], token[7]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat_middle1.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat_middle1), len(token_vecs_cat_middle1[0])))
print(token_vecs_cat[0])

In [ ]:
# Sum the middle 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum_middle1 = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[4:8], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum_middle1.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum_middle1), len(token_vecs_sum_middle1[0])))

In [ ]:
# Concatenate the middle 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat_middle2 = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[8], token[9], token[10], token[11]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat_middle2.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat_middle2), len(token_vecs_cat_middle2[0])))
print(token_vecs_cat[0])

In [ ]:
# Sum the middle 4 hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum_middle2 = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[8:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum_middle2.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum_middle2), len(token_vecs_sum_middle2[0])))

In [ ]:
# Concatenate all hidden layers to create word embeddings.
token_vecs_cat_all = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[0], token[1], token[2], token[3], token[4], token[5], token[6], token[7], token[8], token[9], token[10], token[11]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat_all.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat_all), len(token_vecs_cat_all[0])))
print(token_vecs_cat[0])

In [ ]:
# Sum all hidden layers to create word embeddings.
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum_all = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum_all.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum_all), len(token_vecs_sum_all[0])))

In [ ]:
# Make a single vector to represent the pair of sentences by averaging across tokens.
# `encoded_layers` has shape [12 x 1 x 22 x 768]
sentences_vec = []
# `token_vecs` is a tensor with shape [22 x 768]
token_vecs = encoded_layers[11][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)
for s in sentence_embedding:
    sentences_vec.append(s)
print ("Our final sentence embedding vector of shape:", sentence_embedding.size())
print(sentence_embedding[767])
print(sentence_embedding[-1])
print(f'Shape of sentences vector is: {len(sentences_vec)}')
print(sentences_vec[-1])

In [ ]:
from scipy.spatial.distance import cosine
# Test the similarity of a word with itself.
# For words trained contextually, self-synonymy is less than 1.
similarity = 1 - cosine(token_vecs_cat[0], token_vecs_cat[0])
print(f'Similarity of {tokenized_text[8]} and {tokenized_text[8]} in token_vecs_cat is: {similarity}')
# similarity = 1 - cosine(token_vecs_sum[4], token_vecs_sum[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_sum is: {similarity}')
# similarity = 1 - cosine(token_vecs_cat_first[4], token_vecs_cat_first[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_cat_first is: {similarity}')
# similarity = 1 - cosine(token_vecs_sum_first[4], token_vecs_sum_first[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_sum_first is: {similarity}')
# similarity = 1 - cosine(token_vecs_cat_middle1[4], token_vecs_cat_middle1[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_cat_middle1 is: {similarity}')
# similarity = 1 - cosine(token_vecs_sum_middle1[4], token_vecs_sum_middle1[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_sum_middle1 is: {similarity}')
# similarity = 1 - cosine(token_vecs_cat_middle2[4], token_vecs_cat_middle2[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_cat_middle2 is: {similarity}')
# similarity = 1 - cosine(token_vecs_sum_middle2[4], token_vecs_sum_middle2[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_sum_middle2 is: {similarity}')
# similarity = 1 - cosine(token_vecs_cat_all[4], token_vecs_cat_all[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_cat_all is: {similarity}')
# similarity = 1 - cosine(token_vecs_sum_all[4], token_vecs_sum_all[9])
# print(f'Similarity of {tokenized_text[4]} and {tokenized_text[9]} in token_vecs_sum_all is: {similarity}')

In [7]:
######################################################################################
############## BEGIN TESTING STATIC CONTEXTUAL EMBEDDING CREATION ####################
######################################################################################

In [8]:
def make_vocab(vocab_file):
    # start = timer()
    vocab = []
    # vocab_file = '/home/jupyter/Notebooks/crystal/NLP/MiFace/Python/vocab_files/vocab_checked.txt'
    with open(vocab_file, 'r') as v:
        vocab = v.read().splitlines()
    # end = timer()
    # run_time = end - start
#     print(f'There are {len(vocab)} words in the vocabulary.\n')
#     print(f'It took {run_time} seconds to read the vocabulary file into memory.')
#     print(f'Test word is {vocab[2]}.')
    return vocab

In [9]:
def tokenize_text(vocab):
    tokenized_text = []
    indexed_tokens = []
    for word in vocab:
        # Add the special tokens.
    #     marked_text = "[CLS] " + word + " [SEP]"
        marked_text = word

        # Split the sentence into tokens.
        # tokenized_text = tokenizer.tokenize(marked_text)
        tokenized_text.append(tokenizer.tokenize(marked_text))
#         print(f'Added {tokenized_text[-1]} to the tokenized_text array.')


        # Map the token strings to their vocabulary indeces.
        indexed_tokens.append(tokenizer.convert_tokens_to_ids(tokenized_text[-1]))

        # Display the words with their indeces.
    #     print(f'The word {tokenized_text[-1][1]} is at index {indexed_tokens[-1]}.')
#         for tup in zip(tokenized_text[-1], indexed_tokens[-1]):
#             print('{:<12} {:>6,}'.format(tup[0], tup[1]))
    return tokenized_text, indexed_tokens

In [10]:
def generate_segments_IDs(tokenized_text):
    # Create segment IDs for sentence 1 (there can be a sentence 0 to compare to
    # sentence 1, but we're not doing that).
    # Check that indices and token indices look correct.
    segments_IDs = []
    for i in range(len(tokenized_text)):
        segments_IDs.append([1] * len(tokenized_text[i]))
#     for i in range(len(segments_IDs)):
#         print (segments_IDs[i])
#         print(tokenized_text[i])
    return segments_IDs

In [11]:
def generate_embeddings(indexed_tokens, segments_IDs):
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_IDs])

    # Load pre-trained model (weights)
    model = BertModel.from_pretrained('bert-base-uncased')

    # Put the model in "evaluation" mode, meaning feed-forward operation.
    model.eval()

    # Predict hidden states features for each layer
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors)
#         print('Type of encoded_layers: ', type(encoded_layers))
        # Each layer in the list is a torch tensor.
#         print('Tensor shape for each layer: ', encoded_layers[0].size())

    # Concatenate the tensors for all layers. We use `stack` here to
    # create a new dimension in the tensor.
    token_embeddings = torch.stack(encoded_layers, dim=0)
#     print(token_embeddings.size())

    # Remove dimension 1, the "batches".
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
#     print(token_embeddings.size())

    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)
#     print(token_embeddings.size())
    
    return token_embeddings

In [12]:
def cat_last_four(token_embeddings): 
    # Concatenate the last 4 hidden layers to create contextual embeddings.
    # Stores the token vectors, with shape [22 x 3,072]
    token_vecs_cat_last = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Concatenate the vectors (that is, append them together) from the last four layers.
        # Each layer vector is 768 values, so `cat_vec` is length 3,072.
        cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

        # Use `cat_vec` to represent `token`.
        token_vecs_cat_last.append(cat_vec)

    print ('Shape is: %d x %d' % (len(token_vecs_cat_last), len(token_vecs_cat_last[0])))
    print(token_vecs_cat_last[0])
    
    return token_vecs_cat_last

In [22]:
def cat_first_four(token_embeddings):
    token_vecs_cat_first = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Concatenate the vectors (that is, append them together) from the last 
        # four layers.
        # Each layer vector is 768 values, so `cat_vec` is length 3,072.
        cat_vec = torch.cat((token[0], token[1], token[2], token[3]), dim=0)

        # Use `cat_vec` to represent `token`.
        token_vecs_cat_first.append(cat_vec)

    print ('Shape is: %d x %d' % (len(token_vecs_cat_first), len(token_vecs_cat_first[0])))
    print(token_vecs_cat_first[0])
    
    return token_vecs_cat_first

In [14]:
def get_mean_embedding(token_embeddings):
    mean_embedding = sum(token_embeddings) / len(token_embeddings)
    print(mean_embedding)
    
    return mean_embedding

In [15]:
def reconstruct_tokens(tokenized_text):
    vocab_word = ''
    for i in tokenized_text:
        vocab_word += i.strip('#')
    print(vocab_word)

    return vocab_word

In [16]:
def write_embedding(embeddings_file, vocab_word, contextual_embedding):
    try:
        with open(embeddings_file, 'a') as f:
            f.write(vocab_word)
            for value in contextual_embedding[0]:
                f.write(' ' + str(value.item()))
            f.write('\n')
        print(f'Saved the embedding for {vocab_word}.')
    except:
        print('Oh no! Unable to write to the embeddings file.')

In [ ]:
# if __name__ == "__main__":
# Set up input and output paths.
vocab_file = '/home/jupyter/Notebooks/crystal/NLP/MiFace/Python/vocab_files/vocab_checked.txt'
layer_combining_function = cat_first_four
embeddings_file = os.path.join('/home/jupyter/Notebooks/crystal/NLP/nlp_testing/embeddings_context_vocab', layer_combining_function.__name__ + '.txt')
if os.path.exists(embeddings_file):
    os.remove(embeddings_file)

# Create a list of vocabulary words we want embeddings for.
vocab = make_vocab(vocab_file)

# Tokenize the vocabulary and look up the BERT token indices.
tokenized_text, indexed_tokens = tokenize_text(vocab)

# Generate segment IDs for each token.
segments_IDs = generate_segments_IDs(tokenized_text)

# Generate and write out the contextual embeddings for the vocabulary words.
# Embeddings are saved in a standard format that can be used for calcualting
# the cosine distances between word vectors.
for i in range(len(tokenized_text)):
    # Convert indexed tokens and segments to tensors.
    # Create a BERT model for the tokens.
    # Get the encoded model layers and reshape them.
    token_embeddings = generate_embeddings(indexed_tokens[i], segments_IDs[i])
    print(f'{tokenized_text[i]} has a token embedding of size {token_embeddings.size()}')

    # Extract the contextual embedding for a token.
    contextual_embedding = layer_combining_function(token_embeddings)

    # Write the embedding to a text file, with the vocabulary word prepended.
    vocab_word = reconstruct_tokens(tokenized_text[i])
    # Make sure we've got the correct vocabulary word.
    assert vocab[i] == vocab_word
    write_embedding(embeddings_file, vocab[i], contextual_embedding)

['aback'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.5053,  0.4971,  0.0317,  ..., -0.9494,  0.5799,  0.7537])
aback
Saved the embedding for aback.
['aba', '##shed'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.1385,  0.1992, -0.6109,  ..., -0.1529,  0.0258, -0.1901])
abashed
Saved the embedding for abashed.
['ab', '##hor'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.0943,  0.3695, -1.2897,  ...,  0.1303,  0.2182,  0.5159])
abhor
Saved the embedding for abhor.
['ab', '##hor', '##red'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([ 0.0665,  0.2880, -1.2760,  ...,  0.0154,  0.2874,  0.4358])
abhorred
Saved the embedding for abhorred.
['ab', '##hor', '##rence'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([ 0.0444,  0.3661, -1.2157,  ..., -0.0860,  0.2170,  0.4015])
abhorrence
Saved the embedding 

['aggravated'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.1777,  0.4265, -0.1913,  ..., -0.1204, -0.2994,  0.2673])
aggravated
Saved the embedding for aggravated.
['ag', '##gra', '##vation'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([ 0.0667,  1.0663, -0.3278,  ..., -0.1178,  0.6243,  0.8914])
aggravation
Saved the embedding for aggravation.
['aggression'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.1853,  1.0595, -0.3109,  ..., -0.1024, -0.0157,  0.5554])
aggression
Saved the embedding for aggression.
['aggressive'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.0088,  0.5032, -0.5507,  ..., -0.0484,  0.1403,  0.2706])
aggressive
Saved the embedding for aggressive.
['ag', '##gr', '##ie', '##ve'] has a token embedding of size torch.Size([4, 12, 768])
Shape is: 4 x 3072
tensor([ 0.1614,  0.7959, -0.3026,  ..., -0.2129,  0.8440,  

['angry'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.1087,  0.5733, -0.3362,  ..., -0.0491,  0.3709,  0.4512])
angry
Saved the embedding for angry.
['ang', '##st'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.3287,  0.1654, -0.1497,  ..., -0.2954,  0.2572,  0.5989])
angst
Saved the embedding for angst.
['anguish'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.3491,  0.3474, -0.4197,  ...,  0.1957,  0.3298,  0.0772])
anguish
Saved the embedding for anguish.
['anguish', '##ed'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.2998,  0.3180, -0.4168,  ...,  0.1504,  0.3565,  0.3088])
anguished
Saved the embedding for anguished.
['animated'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.1402,  0.2226, -0.3230,  ...,  0.6311,  0.7730,  0.6633])
animated
Saved the embedding for animated.
['an', '##i

['artificial'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.3631,  0.4375, -0.0695,  ..., -0.8754, -0.4007,  0.5359])
artificial
Saved the embedding for artificial.
['ashamed'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.0583, -0.0833, -0.4251,  ..., -0.1705, -0.1941,  0.0921])
ashamed
Saved the embedding for ashamed.
['aspiring'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.3538, -0.2145, -0.2146,  ..., -0.4136,  0.1431, -0.1170])
aspiring
Saved the embedding for aspiring.
['assert', '##ive'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.6213,  0.4932, -0.4482,  ..., -0.3063, -0.0021,  0.6945])
assertive
Saved the embedding for assertive.
['assert', '##ively'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.6273,  0.3859, -0.3971,  ...,  0.1550, -0.0304,  0.5407])
assertively
Saved the embed

['bear', '##ish'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.3202,  0.2045, -0.1138,  ..., -0.0706,  0.2217,  0.2988])
bearish
Saved the embedding for bearish.
['beat'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([0.4377, 0.1360, 0.2417,  ..., 0.8847, 0.1448, 0.0081])
beat
Saved the embedding for beat.
['beaten'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.0072, -0.1867,  0.2120,  ...,  0.4141,  0.0417,  0.3091])
beaten
Saved the embedding for beaten.
['bed', '##ev', '##iled'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([ 0.6281,  0.1020,  0.0128,  ...,  0.5353, -0.1057, -0.0246])
bedeviled
Saved the embedding for bedeviled.
['be', '##fu', '##ddled'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([ 0.3044,  0.2001, -0.5320,  ...,  0.3145,  0.0343,  0.3894])
befuddled
Saved the embedding for befuddl

...

['res', '##ili', '##ence'] has a token embedding of size torch.Size([3, 12, 768])
Shape is: 3 x 3072
tensor([ 0.5423, -0.1000,  0.3036,  ...,  0.0665,  0.4009,  0.9513])
resilience
Saved the embedding for resilience.
['resistance'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.0683,  0.5221, -0.3828,  ...,  0.5187, -0.4422,  0.4062])
resistance
Saved the embedding for resistance.
['resistant'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.2178,  0.1862, -0.6540,  ..., -0.8682, -0.2249,  0.0070])
resistant
Saved the embedding for resistant.
['resist', '##ent'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.1988,  0.9181, -0.3333,  ..., -0.3997, -0.3588,  1.0457])
resistent
Saved the embedding for resistent.
['resisting'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.1237,  0.9236, -0.4994,  ..., -0.1897, -0.5881,  0.8405])
resisting
S

['sad', '##dened'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.0262,  0.6265,  0.3351,  ...,  0.2080,  0.3951,  0.7532])
saddened
Saved the embedding for saddened.
['sad', '##istic'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([-0.0138,  0.6451,  0.3946,  ...,  0.3908,  0.2973,  0.7125])
sadistic
Saved the embedding for sadistic.
['sadness'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.0093,  0.3940,  0.5799,  ...,  0.0998,  0.2555,  0.4305])
sadness
Saved the embedding for sadness.
['sal', '##acious'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([0.2297, 0.3295, 0.0739,  ..., 0.6823, 0.0734, 0.3116])
salacious
Saved the embedding for salacious.
['saliva', '##ting'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.6350,  0.2834,  0.0767,  ..., -0.5616,  0.2566,  0.2964])
salivating
Saved the embeddi

['searching'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.1335,  0.0057,  0.0112,  ..., -0.2778,  0.4279,  0.5411])
searching
Saved the embedding for searching.
['secretive'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.3127, -0.3362, -0.1284,  ..., -0.2578,  0.2150,  0.1860])
secretive
Saved the embedding for secretive.
['secretive', '##ly'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.1245, -0.0758,  0.0838,  ..., -0.0996,  0.3222,  0.4213])
secretively
Saved the embedding for secretively.
['secure'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([-0.0189,  0.0635, -0.0660,  ..., -0.3796,  0.3496,  0.7913])
secure
Saved the embedding for secure.
['se', '##date'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.0122, -0.3124, -0.4727,  ...,  0.0544,  0.1101,  0.8792])
sedate
Saved the embedding fo

['sick', '##en'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.2728,  0.5179,  0.2745,  ..., -0.4409,  0.1305, -0.0965])
sicken
Saved the embedding for sicken.
['sick', '##ened'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.1323,  0.5583,  0.1762,  ..., -0.2202,  0.1342, -0.0661])
sickened
Saved the embedding for sickened.
['sigh'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.0406,  0.4564,  0.6383,  ..., -0.2941, -0.2850,  0.2434])
sigh
Saved the embedding for sigh.
['silenced'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.5748,  0.8378, -0.5147,  ...,  0.0403, -0.0738,  0.2835])
silenced
Saved the embedding for silenced.
['silent'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.7121,  0.5796, -0.3927,  ..., -0.1244,  0.2935,  0.1680])
silent
Saved the embedding for silent.
['si', '##llin', 

['smooth'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.2714, -0.0560, -0.0799,  ...,  0.0659,  0.1826, -0.1712])
smooth
Saved the embedding for smooth.
['smug'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.3021, -0.0421,  0.4123,  ...,  0.1663,  0.0923,  0.2920])
smug
Saved the embedding for smug.
['smug', '##ness'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([0.2697, 0.0788, 0.4565,  ..., 0.2410, 0.1085, 0.6001])
smugness
Saved the embedding for smugness.
['snake'] has a token embedding of size torch.Size([1, 12, 768])
Shape is: 1 x 3072
tensor([ 0.8965,  0.0487, -0.0098,  ...,  0.4522,  0.3907, -0.2765])
snake
Saved the embedding for snake.
['snap', '##py'] has a token embedding of size torch.Size([2, 12, 768])
Shape is: 2 x 3072
tensor([ 0.4640, -0.6222,  0.2072,  ...,  0.3994,  0.4374, -0.4563])
snappy
Saved the embedding for snappy.
['s', '##nar', '##ky'] has a tok